### prompt template
- prompt template : 프롬프트(모델입력값) 의 함수화
- 어떠한 서비스 제작시 사용자의 부담을 덜어준다.

In [11]:
import json
import os
with open("api_keys.json") as f:
    os.environ["OPENAI_API_KEY"] = json.load(f)["openai"]

In [12]:
from langchain_openai import ChatOpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

gpt4o = ChatOpenAI(model_name="gpt-4o-mini",
                   streaming=True,
                   callbacks=[StreamingStdOutCallbackHandler()],
                   temperature=1,
                   max_tokens=100)

answer = gpt4o.invoke("hi")

Hello! How can I assist you today?

In [20]:
from langchain.schema import AIMessage, HumanMessage, SystemMessage

messages = [
    SystemMessage("You are a kindergarten teacher. Please answer as if you were speaking to a little baby."),
    HumanMessage("안녕! 하쪠여 뜌뜌땨"),
]

responses = gpt4o.invoke(messages)

안녕, 귀여운 아가야! 😊 어떻게 지내고 있니? 오늘도 재미있는 것들 많이 했니? 행복한 하루 보내자! 💖🌈

### Prompt Template

In [47]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate

system_prompt = PromptTemplate.from_template("You are a {profession}. Please answer as if you were speaking to a {target}")

# format_prompt 로 설정을 해준다. 방법 1
system_prompt_value = system_prompt.format_prompt(profession = "expert job planner", target = "job seeker in ai industry")

messages = [
    # SystemMessage(system_prompt_value.to_string()), # 방법 1
    SystemMessage(system_prompt.format(profession = "교수님", target = "대학생")), # 방법 2
    HumanMessage("당신의 직업은 무엇입니까?"),
]
gpt4o.invoke(messages);

안녕하세요! 저는 교수님입니다. 주로 학생들에게 지식을 전달하고, 연구를 수행하며, 학문적 발전에 기여하는 일을 하고 있습니다. 여러분의 학습을 돕고, 궁금한 점에 대해 함께 고민하는 것은 제 역할 중 하나입니다. 혹시 어떤 과목에 대해 질문이 있으신가요?

### Chatting Promplt Template

In [ ]:
from langchain.prompts import (ChatPromptTemplate,
                               PromptTemplate,
                               SystemMessagePromptTemplate,
                               AIMessagePromptTemplate,
                               HumanMessagePromptTemplate)

from langchain.schema import AIMessage, HumanMessage, SystemMessage

from langchain_openai import ChatOpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

gpt4o = ChatOpenAI(model_name="gpt-4o-mini",
                   streaming=True,
                   callbacks=[StreamingStdOutCallbackHandler()],
                   temperature=0.5,
                   max_tokens=1000)

system_message_prompt = SystemMessagePromptTemplate.from_template("역할 은{system_prompt} 입니다. 대답항목 {N__} 이하")
human__message_prompt = HumanMessagePromptTemplate.from_template("질문 : {human_prompt}")
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt,human__message_prompt])

answer = gpt4o.invoke(chat_prompt.format_prompt(system_prompt= "철저한 계획 기획자",
                                                N__ = "5",
                                                human_prompt = "완전한 로봇 제작 계획"));

완전한 로봇 제작 계획은 다음과 같은 단계로 구성됩니다:

1. **목표 설정**: 로봇의 용도와 기능 정의 (예: 가정용, 산업용, 교육용 등).

2. **설계**: 
   - 기계적 구조 설계 (CAD 소프트웨어 사용).
   - 전자 회로 설계 (센서, 모터, 마이크로컨트롤러 등).

3. **부품 선정**: 
   - 필요한 부품 목록 작성.
   - 부품 공급처 조사 및 구매.

4. **조립**: 
   - 기계적 조립 및 전자 회로 연결.
   - 소프트웨어 개발 및 로봇 제어 시스템 구축.

5. **테스트 및 개선**: 
   - 기능 테스트 및 성능 평가.
   - 문제점 수정 및 최적화.

각 단계에서의 세부 계획과 일정 설정이 중요합니다.

### template for few-shot

- 여러 결과물을 예시로 내는 few-shot 프롬프팅

In [159]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate

examples = [
    {
        "question" : "로봇 이미지 생성을 위한 프롬프트를 만들어줘",
        "answer" : "로봇, 공상과학, 우주, 지구"
    },
    {
        "question" : "동화 이미지 생성을 위한 프롬프트를 만들어줘?",
        "answer" : "산, 토끼, 거북이, 달리기"
    },
]

In [160]:
examples_prompt = PromptTemplate(input_variables=["question","answer"], template="question : {question} \nanswer: {answer}")

In [161]:
examples_prompt.format(**examples[0])

'question : 로봇 이미지 생성을 위한 프롬프트를 만들어줘 \nanswer: 로봇, 공상과학, 우주, 지구'

In [ ]:
few_prompt = FewShotPromptTemplate(
    examples = examples, # 참고 예시 대상
    example_prompt = examples_prompt, # 표현 정의 템플릿
    suffix = "question : {input}이미지 생성을 위한 프롬프트를 만들어줘", # 실제 사용 템플릿
    input_variables = ["input"]
)
answer = gpt4o.invoke(few_prompt.format(input = "멋진 뒷골목에서 날아다니는 새")) # 원하는 형식의 템플릿이 생성됨

answer: 멋진 뒷골목, 화려한 그래피티, 날아다니는 새, 햇살, 도시 풍경

In [177]:
gpt4o.invoke("멋진 뒷골목에서 날아다니는 새이미지를 위한 프롬프트를 생성해줘");

물론입니다! 멋진 뒷골목에서 날아다니는 새 이미지를 위한 프롬프트는 다음과 같습니다:

"화창한 오후, 고풍스러운 벽돌 건물과 화려한 그래피티로 가득한 뒷골목. 그 사이로 다양한 색깔의 새들이 자유롭게 날아다니며, 햇살에 반짝이는 깃털이 아름답게 빛난다. 뒷골목의 작은 카페 테라스에는 사람들이 앉아 새들을 바라보며 여유로운 시간을 보내고 있다. 새들의 날갯짓이 생동감 넘치는 장면을 만들어내는 순간을 포착한 이미지."

이 프롬프트를 사용하여 원하는 이미지를 생성해보세요!

In [178]:
print(answer)

content='answer: 멋진 뒷골목, 화려한 그래피티, 날아다니는 새, 햇살, 도시 풍경' additional_kwargs={} response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0705bf87c0'} id='run-10767110-e047-4a43-9e2d-25a16ab45e71-0'


### Example Selector(동적 Few-shot)
- `SemanticSimilarityExampleSelector` : 의미의 유사도를 계산하여 적절한 예시를 선택
- `OpenAIEmbeddings()` : 텍스트의 벡터화 (임베딩화)
- `Chroma` : 벡터화하여 저장

In [ ]:
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector # 사용자 입력과, 예시 샘플의 유사도 비교
from langchain.vectorstores import Chroma # 임베딩 된 벡터를 비교
from langchain.embeddings import OpenAIEmbeddings # 문자의 임베딩 (수치화)
from langchain.prompts import FewShotPromptTemplate, PromptTemplate

example_prompt = PromptTemplate(
    input_variables=["input","output"],
    template="input = {input}, output = {output}",
)

examples = [
    {"input" : "happy", "output" : "unhappy"},
    {"input" : "plane", "output" : "ship"},
    {"input" : "big plane", "output" : "small plane"}
]

In [21]:
example_selector = SemanticSimilarityExampleSelector.from_examples( # 의미적 유사도를 계산하여
    examples=examples, # 예시s
    embeddings=OpenAIEmbeddings(), # 임베딩 벡터라이제이션
    vectorstore_cls=Chroma, # 벡터 저장 소
    k=1 # 가장 유사한 상위 k 개
)

similar_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="give me an opposite word",
    suffix="input is {word}, and output is",
    input_variables = ["word"],
)

In [50]:
print(similar_prompt.format(word="manpower"))

give me an opposite word

input = plane, output = ship

input is manpower, and output is


In [55]:
gpt4o.invoke(similar_prompt.format(word="manpower"));

output = automation

### out parser를 이용하여 출력값 조정 (서비스 결합)

In [75]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate

output_parser = CommaSeparatedListOutputParser()
output_parser.get_format_instructions()

'Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'

In [ ]:
format_instuctions = output_parser.get_format_instructions()

prompt = PromptTemplate(
    template="give me 5 {subject} recommends. \{format_instuctions}",
    input_variables=["subject"], # 사용자 입력부분
    partial_variables={"format_instructions": format_instuctions}
)

In [111]:
input_prompt = prompt.format(subject="nice place to get a job",
                             format_instuctions = format_instuctions)
input_prompt

'give me 5 nice place to get a job recommends. \\Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'

In [112]:
gpt4o.invoke(input_prompt);

Google, Microsoft, IBM, Amazon, Salesforce